In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import xlrd
import plotly.graph_objs as go
import functools
pd.set_option('display.max_rows', 150)

In [ ]:
df = pd.read_excel("2003-2016 Seed Potato Cert data v20191204_NO FL lines_Rioux 5AUG2020.xlsx", sheet_name="2003-2016 Seed Potato Cert")

In [ ]:
df.shape

In [ ]:
df.head()

### Data Validation

#### Inspection day

Hypothesis: **DASP1** should be smaller than **DASP2**

In [ ]:
df.loc[df["DAPS1"] > df["DAPS2"], df.columns.str.contains("DA")]

#### Source Year

Hypothesis: **CY** should be larger than or equal to **S_YR** <br>

-CY: year entered certification program <br>
-S_YR: source year 

In [ ]:
df[(df["CY"] <df["S_YR"]) | (df["CY"] <df["winter_S_YR"])]

#### Check one-to-one relationship between source grower and source grower code

Hypothesis: One source grower should only have one source grower code

In [ ]:
# One source code correspond 5 different source grower

grw_count = df[["S_GRW","S_GCODE"]].groupby("S_GCODE").apply(lambda x: x["S_GRW"].nunique())
grw_count[grw_count > 2]

In [ ]:
winter_grw_count = df[["winter_S_GRW","winter_S_GCODE"]].groupby("winter_S_GCODE").apply(lambda x: x["winter_S_GRW"].nunique())
winter_grw_count[winter_grw_count > 2]

In [ ]:
grw_count = df[["S_GRW","S_GCODE"]].groupby("S_GCODE").apply(lambda x: x["S_GRW"].nunique())
grw_count[grw_count > 2]

In [ ]:
# It seems like these 5 source grower are actually one source grower in different format
df[df["S_GCODE"] == 243]["S_GRW"].value_counts()

In [ ]:
df[df["winter_S_GCODE"] == 243]["winter_S_GRW"].value_counts()

In [ ]:
# Standardize name with the most common one: 'Felix Zeloski Farms-Eagle River'
prob_name = df[df["S_GCODE"] == 243]["S_GRW"].unique().tolist()
prob_name
df["S_GRW"] = df["S_GRW"].apply(lambda x: 'Felix Zeloski Farms-Eagle River' if x in prob_name else x)
df["winter_S_GRW"] = df["winter_S_GRW"].apply(lambda x: 'Felix Zeloski Farms-Eagle River' if x in prob_name else x)

# Double check after fixing the problem
grw_count = df[["S_GRW","S_GCODE"]].groupby("S_GCODE").apply(lambda x: x["S_GRW"].nunique())
grw_count[grw_count > 2]

In [ ]:
winter_grw_count = df[["winter_S_GRW","winter_S_GCODE"]].groupby("winter_S_GCODE").apply(lambda x: x["winter_S_GRW"].nunique())
winter_grw_count[winter_grw_count > 2]

In [ ]:
df["S_GRW"].value_counts()

In [ ]:
# Three source growers have 2 source grower code

gcode_count = df[["S_GRW","S_GCODE"]].groupby("S_GRW").apply(lambda x: x["S_GCODE"].nunique())
gcode_count[gcode_count > 1]



In [ ]:
# Look into these three problematic grower
prob_grower = gcode_count[gcode_count > 1].index.tolist()
prob_grower

for grower in prob_grower:
    print(str(grower) +":" + str(df[df["S_GRW"] == grower]["S_GCODE"].unique()))
    print()
# prob_grw = gcode_count[gcode_count > 1].index

#### Possible redundant info for summer and winter

Summer info and winter info should match for source info

In [ ]:
summer_columns = ["CERT_N",
"SNAME",
"GCODE",
"VARIETY",
"S_GRW",
"S_G",
"S_YR",
"S_GCODE",
"S_STATE"]

winter_columns = ["winter_{}".format(x) for x in summer_columns]
winter_columns

combined_columns =[]
for i in range(len(summer_columns)):
    combined_columns.append(summer_columns[i])
    combined_columns.append(winter_columns[i])
    
combined_columns

In [ ]:
df[combined_columns]

##### Check the reason for mismatch

Some of mismatch result from missing value

Solution: fill the missing value by its equivalent

In [ ]:
conditions = [(df.loc[:, combined_columns[i]] != df.loc[:,combined_columns[i+1]]) for i in range(0,len(combined_columns),2)]
conditions

df.loc[conditions[0] | conditions[1] | conditions[2] | conditions[3] | conditions[4] | conditions[5] | conditions[6] | conditions[7] | conditions[8],combined_columns
      ] 

Fill na value in either summer or winter columns by the other one

In [ ]:
for i in range(0, len(combined_columns),2):
    df[combined_columns[i]] = df[combined_columns[i]].fillna(df[combined_columns[i+1]])
    df[combined_columns[i]] = df[combined_columns[i]].mask(df[combined_columns[i]]==0).fillna(df[combined_columns[i+1]])

for i in range(1, len(combined_columns),2):
    df[combined_columns[i]] = df[combined_columns[i]].fillna(df[combined_columns[i-1]])
    df[combined_columns[i]] =df[combined_columns[i]].mask(df[combined_columns[i]]==0).fillna(df[combined_columns[i-1]])


##### Mismatch Analysis

- Reason1: Typo 
- Reason2: full name and shorthand notation in variety
- Reason3: Extra comma (source grower)
 

In [ ]:
new_conditions = [(df.loc[:, combined_columns[i]] != df.loc[:,combined_columns[i+1]]) for i in range(0,len(combined_columns),2)]
new_conditions

df.loc[new_conditions[0] | new_conditions[1] | new_conditions[2] | new_conditions[3] | new_conditions[4] | new_conditions[5] | new_conditions[6] | new_conditions[7] | new_conditions[8],combined_columns
      ] 

In [ ]:
# Number of mismatch after filling missing value

for i in range(len(new_conditions)):
    print(combined_columns[i*2] + ":" + str(len(df.loc[new_conditions[i], combined_columns])))

In [ ]:
df.loc[df["S_G"] != df["winter_S_G"], combined_columns]

In [ ]:
df[["S_G"]].value_counts()

Fix the problem of S_G:

- Upper case and lower case (CETS and Cets) Cets => CETS
- Plural and singular forms (Nilson Farms & Nilson Farm)
- Spelling error (Schroeder Farm, Schroder Farms) Schroder Farms => Schroeder Farm
- Order (J Gallenberg, Gallenberg J)

In [ ]:
df["winter_S_G"] = df["winter_S_G"].apply(lambda x: "CETS" if x == "Cets" else x)

df["winter_S_G"] = df["winter_S_G"].apply(lambda x: "Schroeder Farm" if x == "Schroder Farms" else x)

df["winter_S_G"] = df["winter_S_G"].apply(lambda x: "J Gallenberg" if x == "Gallenberg J" else x)


In [ ]:
df.loc[df["S_G"] != df["winter_S_G"], combined_columns]

Fix the problem of S_STATE: 

- change PEI to PE

In [ ]:
df["S_STATE"] = df["S_STATE"].apply(lambda x:"PE" if x == "PEI" else x)
df["S_STATE"]

Fix the problem of S_YR: 

- change 1072 (Typo) to 2006

In [ ]:
df["S_YR"] = df["S_YR"].apply(lambda x:2006 if x == 1072 else x)
df["S_YR"]

In [ ]:
df.loc[df["S_YR"] != df["winter_S_YR"], combined_columns]

In [ ]:
df.loc[df["winter_S_STATE"] == "PE", combined_columns]

In [ ]:
def conjunction(*conditions):
    return functools.reduce(np.logical_or, conditions)

condition
np.lo

source_info[conjunction(condition[0], condition[1])]

In [ ]:
df.loc[:, (df.columns.str.contains("SR2")) | (df.columns.str.contains("SR1")) ]


In [ ]:
frequent_state = df["S_STATE"].value_counts()[:8].index.to_list()
frequent_state

In [ ]:
df.columns[df.columns.str.contains("SR1")]

In [ ]:
target_virus = ['SR1_ST','SR1_MIX',"SR1_LR"]

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
df[df["S_STATE"].isin(frequent_state)].groupby("S_STATE").mean()[target_virus].plot(kind = "barh", ax = ax)
ax.tick_params(axis='x', rotation=0)
ax.set_title("Virus across state")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()

plt.savefig("Virus_across_state")

In [ ]:
frequent_variety = df["VARIETY"].value_counts()[:15].index.tolist()
frequent_variety

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
df[df["VARIETY"].isin(frequent_variety)].groupby("VARIETY").mean()[target_virus].plot(kind = "barh", ax = ax)
ax.tick_params(axis='x', rotation=0)
ax.tick_params(axis="y", rotation = 0)
ax.set_title("Virus across potato variety")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig("Virus_across_potato_variety")